In [1]:
import sys, os
path_set = False
if path_set is False:
    sys.path.append('../')
    os.chdir('../')
    path_set = True

import constant
from datasets import data_loader as dl
from models.model_fectory import MDANet, load_model
import read_config
from utils import get_logger

import torch
import time
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import copy
import argparse
import shutil

from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [2]:
# setting path
config = read_config.read('conf_files/config_238_serenext50')
test_case = config['name']
class_number = config['class_number']
extractor = config['extractor']
number_worker = config['number_workers']
saved_model_path = os.path.join(constant.logs_root, test_case, 'best_model.pt')
submit_output = os.path.join(constant.logs_root, test_case, 'submit.txt')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# restoring model
mdan = load_model('mdan', class_number, 3, extractor=extractor).to(device)
checkpoint = torch.load(saved_model_path)
mdan.load_state_dict(checkpoint['model_state_dict'])
mdan.eval()

/tf/notebooks/cv_homework/MDAN/models/model_fectory.py:90: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  torch.nn.init.xavier_uniform(m.weight)


MDANet(
  (feature_extractor): SENetExtractor(
    (layer0): Sequential(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu1): ReLU(inplace)
      (pool): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    )
    (layer1): Sequential(
      (0): SEResNeXtBottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    

In [4]:
test_dataset = dl.load_data('test', resize = (224,224))
test_dataloader = DataLoader(test_dataset, batch_size=1,shuffle=False, num_workers=number_worker)

predictions = []

print('Predicting the test images...')
for inputs, img_path in test_dataloader:
    inputs = inputs.to(device)

    # forward
    with torch.set_grad_enabled(False):
        outputs = mdan.inference(inputs)
        _, preds = torch.max(outputs, 1)
        predictions.append((img_path, preds.item()))
        
result = []
for (image_path, pred) in predictions:
    result.append('{},{}\n'.format(image_path[0], pred))

Predicting the test images...


In [5]:
with open(submit_output, 'w+') as fout:
    fout.write('image_name,label\n')
    for line in result:
        fout.write(line)